# TD 5 : Modèle relationnel, analyse d'incidents dans le transport aérien

In [1]:
import pyensae
pyensae.download_data("tp_2a_5_compagnies.zip")

    downloading of  http://www.xavierdupre.fr/enseignement/complements/tp_2a_5_compagnies.zip  to  tp_2a_5_compagnies.zip
    unzipped 

 Cities.csv  to  .\Cities.csv
    unzipped  Compagnies.csv  to  .\Compagnies.csv
    unzipped  Crews.csv  to  .\Crews.csv
    unzipped  Crews_planes_habilitation.csv  to  .\Crews_planes_habilitation.csv
    unzipped  Data_exemple.csv  to  .\Data_exemple.csv
    unzipped  Data_exemple_2.csv  to  .\Data_exemple_2.csv
    unzipped  

Data_exemple_3.csv  to  .\Data_exemple_3.csv
    unzipped  Data_exemple_4.csv  to  .\Data_exemple_4.csv
    unzipped  data_model.txt  to  .\data_model.txt
    unzipped  data_model_to_csv.py  to  .\data_model_to_csv.py
    unzipped  enonce.txt  to  .\enonce.txt
    unzipped  Exemple_arborescent.xml  to  .\Exemple_arborescent.xml
    unzipped 

 Flights.csv  to  .\Flights.csv
    unzipped  Incident.csv  to  .\Incident.csv
    unzipped  Motors.csv  to  .\Motors.csv
    unzipped  Motor_models.csv  to  .\Motor_models.csv
    unzipped  Planes.csv  to  .\Planes.csv
    unzipped  Plane_models.csv  to  .\Plane_models.csv


['.\\Cities.csv',
 '.\\Compagnies.csv',
 '.\\Crews.csv',
 '.\\Crews_planes_habilitation.csv',
 '.\\Data_exemple.csv',
 '.\\Data_exemple_2.csv',
 '.\\Data_exemple_3.csv',
 '.\\Data_exemple_4.csv',
 '.\\data_model.txt',
 '.\\data_model_to_csv.py',
 '.\\enonce.txt',
 '.\\Exemple_arborescent.xml',
 '.\\Flights.csv',
 '.\\Incident.csv',
 '.\\Motors.csv',
 '.\\Motor_models.csv',
 '.\\Planes.csv',
 '.\\Plane_models.csv']

* [Question 1 : Question préliminaires, taux d'incident](#question1)
* [Question 2 : Probabilité d'incident par facteur](#question2)
* [Question 3 : Régression linéaire](#question3)
* [Question 4 : Iberia VS Lufthansa](#question4)
* [Question 5 : Facteurs explicatifs](#question5)


On travaille sur un modèle de donnée concernant des incidents dans le transport aérien, et on cherche à en déterminer les causes.
La prédiction des incidents est un domaine important dans le transport aérien, car on ne possède pas de donnée statistiquement significative sur les accidents (et on ne souhaite pas en obtenir ...).

On possède un modèle de donnée avec 10 tables et les champs suivants :

    Incident : Flight_id
    Flights  : Id ; Plane_id (key in Planes) ; Crew_id (key in Crews) ; Departure_id (key in Cities) ; Arrival_id (key in Cities) ; Date
    Crews    : Id ; Captain_age
    Crews_planes_habilitation : Crew_id (key in Crew) ; Plane_model_id (key in Plane_models)
    Planes       : Id ; Immatriculation ; Plane_model_id (key in Plane_models) ; Motor_id (key in Motors) ; Nb_hours
    Plane_models : Id ; Constructor ; Model_name
    Motors       : Id ; Motor_model_id (key in Motor_models ) ; Nb_hours
    Motor_models : Id ; Name ; Manufacturer
    Compagnies   : Id ; Name ; Country
    Cities       ; Id ; Name ; Country

On possède donc un certain nombre de facteurs explicatifs potentiel pour ces incidents : le modèle du moteur, de l'avion, leur nombre d'heure d'utilisation respectif, l'équipage, l'age du capitaine, les villes de départ, d'arrivée, la compagnie, etc ...

Commencer par regarder un peu les données, leur nombre, leur type, etc ...

In [2]:
import os
import pandas
df_Incident = pandas.DataFrame.from_csv( 'Incident.csv', sep = ';' )
df_Flights  = pandas.DataFrame.from_csv( 'Flights.csv', sep = ';' )
df_Crews    = pandas.DataFrame.from_csv( 'Crews.csv', sep = ';' )
df_Crews_planes_habilitation = pandas.DataFrame.from_csv( 'Crews_planes_habilitation.csv', sep = ';' )
df_Planes       = pandas.DataFrame.from_csv( 'Planes.csv', sep = ';' )
df_Plane_models = pandas.DataFrame.from_csv( 'Plane_models.csv', sep = ';' )
df_Motors       = pandas.DataFrame.from_csv( 'Motors.csv', sep = ';' )
df_Motor_models = pandas.DataFrame.from_csv( 'Motor_models.csv', sep = ';' )
df_Compagnies   = pandas.DataFrame.from_csv( 'Compagnies.csv', sep = ';' )
df_Cities       = pandas.DataFrame.from_csv( 'Cities.csv', sep = ';' )

In [3]:
df_Incident.head(5)

""
Flight_id
2
11
21
30
33


In [9]:
df_Flights.head(5)

,Plane_id,Crew_id,Departure_id,Arrival_id,Date
Id,,,,,
0,40,85,7,0,"datetime.date(2013, 2, 20)"
1,67,6,16,4,"datetime.date(2013, 1, 9)"
2,57,67,1,18,"datetime.date(2013, 8, 7)"
3,41,69,16,14,"datetime.date(2013, 1, 10)"
4,60,24,4,7,"datetime.date(2013, 9, 26)"


In [ ]:
df_Crews.head(5)

In [ ]:
df_Crews_planes_habilitation.head(5)

In [ ]:
df_Planes.head(5)

In [ ]:
df_Plane_models.head(5)

In [ ]:
df_Motors.head(5)

In [5]:
df_Motor_models.head(5)

,Name,Manufacturer
Id,,
0,SNECMA-123,SNECMA
1,SNECMA-246,SNECMA
2,GE-247,GE
3,GE-656,GE
4,ROLLS ROYCE-345,ROLLS ROYCE


In [ ]:
df_Compagnies.head(5)

In [4]:
df_Cities.head(5)

,Name,Country
Id,,
0,Bombay,Inde
1,New York,USA
2,Tokyo,Japon
3,Paris,France
4,Francfort,Allemagne


Bien sûr, toutes les informations ne sont pas dans les tables telles quelles, il faudra faire principalement des jointures et des groupby pour obtenir les informations que l'on souhaite. Pour obtenir une table contenant les vols avec une colonne "est_incident", il faut faire :

In [17]:
import numpy as np
try:
  df_Flights.reset_index( inplace = True )
  df_Incident.reset_index( inplace = True )
except Exception:
  pass
  ## On suppose que cela vient du fait que les index ont déjà été remis à zéros
df_Flight_Incident = pandas.merge( df_Flights, df_Incident, left_on = "Id", right_on = "Flight_id", how="outer" )
df_Flight_Incident["Is_incident"] = np.isnan( df_Flight_Incident["Flight_id"] ) == False
df_Flight_Incident.head(5)

,level_0,index_x,Id,Plane_id,Crew_id,Departure_id,Arrival_id,Date,index_y,Flight_id,Is_incident
0,0,0,0,40,85,7,0,"datetime.date(2013, 2, 20)",NaN,NaN,False
1,1,1,1,67,6,16,4,"datetime.date(2013, 1, 9)",NaN,NaN,False
2,2,2,2,57,67,1,18,"datetime.date(2013, 8, 7)",0,2,True
3,3,3,3,41,69,16,14,"datetime.date(2013, 1, 10)",NaN,NaN,False
4,4,4,4,60,24,4,7,"datetime.date(2013, 9, 26)",NaN,NaN,False


On peut aussi faire des statistiques par ville de départ ...

In [20]:
df_Flight_Incident.groupby( "Departure_id" )["Is_incident"].mean()

Departure_id
0               0.146789
1               0.192308
2               0.220930
3               0.222222
4               0.206107
5               0.218391
6               0.180180
7               0.261682
8               0.120879
9               0.131868
10              0.141176
11              0.184466
12              0.097087
13              0.204082
14              0.174312
15              0.186916
16              0.206897
17              0.272727
18              0.189474
19              0.211538
Name: Is_incident, dtype: float64

<h3 id="question1">Question 1 : Question préliminaires, taux d'incident</h3>

A vous de jouer. Commencer par calculer le taux d'incident global.

<h3 id="question2">Question 2 : Calcul de taux</h3>
  
Calculer le taux d'incidents par :  
  => Compagnies  
  => Destination  
  => Equipage  
  => Constructeur d'appareil (Boeing, Airbus ou Bombardier)  
  => Modèle d'appareil (A320, A380, B737, etc ...)  
  => Moteurs  

<h3 id="question3">Question 3 : Régression linéaire</h3>
    
Pouvez-vous raisonnablement faire une régression linéaire sur le nombre d'heure de l'appareil ? Du moteur ? Pourquoi ?

<h3 id="question4">Question 4 : Iberia VS Lufthansa</h3>
    
Pouvez-vous dire si, toute chose égale par ailleurs, un avion d'Iberia a plus ou moins de chance d'avoir un incident qu'un avion de la Lufthansa ?

<h3 id="question5">Question 5 : Facteurs explicatifs</h3>
    
Essayer de déterminer quels sont les facteurs explicatifs. Lesquels vous paraissent statistiquement significatifs ? Y-a-t-il des corrélations indirectes ? 